In [13]:
import torch
import cv2
from torchvision import models, transforms

In [20]:
model = models.segmentation.deeplabv3_mobilenet_v3_large(pretrained=True).eval()

C:\Users\fluri\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\fluri\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_mobilenet_v3_large-fc3c493d.pth" to C:\Users\fluri/.cache\torch\hub\checkpoints\deeplabv3_mobilenet_v3_large-fc3c493d.pth
100%|██████████| 42.3M/42.3M [00:15<00:00, 2.89MB/s]


In [21]:
def segment(frame, orig_dims):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    ])

    input_tensor = preprocess(frame)
    input_batch = input_tensor.unsqueeze(0)

    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')
    with torch.no_grad():
        output = model(input_batch)['out'][0]
    output_predictions = output.argmax(0)

    mask = output_predictions.byte().cpu().numpy()
    mask = cv2.resize (mask, (orig_dims[1], orig_dims[0]))
    return mask

In [22]:
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
orig_dims = (frame_height, frame_width)

In [23]:
while(True):
    ret, frame = cap.read()
    if not ret:
        break
    mask = segment(frame, orig_dims)
    frame[mask == 0] = [0, 0, 0]
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    

In [24]:
cap.release()
cv2.destroyAllWindows()